<span style="font-size:large;">
<div style="text-align: right">IM-UH 1511 <b>Introduction to Digital Humanities</b><br/>
Student name: <span style="color:blue"><b>X XX</b></span><br/>
Grade: <span style="color:red"><b></b></span><br/><br/></div>

## <span style="color:red">HOMEWORK 5a</span>
# <span style="color:green">Twitter Search Mining

In [1]:
import time
start_time = time.perf_counter()
import pandas as pd
import json 
from datetime import datetime, timedelta
import twitter ## pip install --user python-twitter
import os
import twitter
from bs4 import BeautifulSoup
from urllib.parse import quote, unquote
import requests

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 
warnings.filterwarnings("ignore", category=UserWarning) 
warnings.simplefilter('ignore')

In [2]:
def tweet_cleaner(status):
    payload = {}
    payload['screen_name'] = status['user']['screen_name']
    payload['created'] = pd.to_datetime(status['created_at'])
    payload['retweets'] = status['retweet_count']
    payload['favorites'] = status['favorite_count']
    payload['id'] = status['id']
    payload['reply_screen_name'] = status['in_reply_to_screen_name']
    payload['reply_id'] = status['in_reply_to_status_id']
    payload['source'] = BeautifulSoup(status['source']).text
    payload['lang'] = status['lang']
    
    if status['place']:
        payload['place'] = status['place']['country']
    else:
        payload['place'] = None

    if len(status['entities']['user_mentions']) > 0:
        payload['user_mentions'] = '; '.join([m['screen_name'] for m in status['entities']['user_mentions']])
    else:
        payload['user_mentions'] = None

    if len(status['entities']['hashtags']) > 0:
        payload['hashtags'] = '; '.join([h['text'] for h in status['entities']['hashtags']])
    else:
        payload['hashtags'] = None

    # If an account retweets another account, we should store that information
    if 'retweeted_status' in status:
        rt_status = status['retweeted_status']
        if 'extended_tweet' in rt_status:
            payload['text'] = rt_status['extended_tweet']['full_text']
            if len(rt_status['extended_tweet']['entities']['hashtags']) > 0:
                payload['hashtags'] = '; '.join([h['text'] for h in rt_status['extended_tweet']['entities']['hashtags']])
            else:
                payload['hashtags'] = None
        else:
            try:
                payload['text'] = rt_status['text']
            except:
                payload['text'] = rt_status['full_text']
            if len(rt_status['entities']['hashtags']) > 0:
                payload['hashtags'] = '; '.join([h['text'] for h in rt_status['entities']['hashtags']])
            else:
                payload['hashtags'] = None
        payload['is_retweet'] = True
        payload['retweeted_screen_name'] = rt_status['user']['screen_name']
        payload['retweeted_created'] = rt_status['created_at']
        payload['retweeted_source'] = BeautifulSoup(rt_status['source'], "lxml").text
        
    else:
        if status['truncated']:
            payload['text'] = status['extended_tweet']['full_text']
        else:
            try:
                payload['text'] = status['text']
            except:
                payload['text'] = status['full_text']
        payload['is_retweet'] = False
        payload['retweeted_screen_name'] = False
        payload['retweeted_created'] = False
        payload['retweeted_source'] = False

    return payload

In [3]:
# # My key information file is a text file ('twitter_keys.json') of the following form:

# {
#     "consumer_key":"API key",
#  "consumer_secret":"API secret key",
#  "access_token_key":"Access token",
#  "access_token_secret":"Access token secret"
# }

In [4]:
# Load my key information from disk

with open('twitter_keys.json','r') as f:
    twitter_keys = json.load(f)

# Authenticate with the Twitter API using the twitter_keys dictionary
# The "tweet_mode='extended' allows us to see the full 280 characters in tweets

api = twitter.Api(**twitter_keys,tweet_mode='extended')

In [5]:
# # Alternatively, you can just enter your keys directly into the Api function:

# api = twitter.Api(consumer_key = 'API key',
#                   consumer_secret = 'API secret key',
#                   access_token_key = 'Access token',
#                   access_token_secret = 'Access token secret',
#                   tweet_mode='extended')

### Search API

Twitter's [search API](https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets) provides an endpoint to search for tweets matching a query for terms, accounts, hashtags, language, locations, and date ranges. This API endpoint has a rate limit of 180 requests per 15-minute window with 100 statuses per request: or 18,000 statuses per window or 72,000 statuses per hour.

You can explore some of the search functionality through Twitter's [advanced search interface](https://twitter.com/search-advanced). Note that the [standard search API](https://developer.twitter.com/en/docs/tweets/search/overview/standard) only provides a limited access to sample of tweets in the past 7 days, you'll need to pay more to access [historical APIs](https://developer.twitter.com/en/docs/tutorials/choosing-historical-api.html).

In [6]:
search_term = "Louvre"
no_tweets = 2000 # This number should be less than 18000! [read above]

search_tweets = []

while True:
    # Get the first set of tweets
    if len(search_tweets) == 0:
        query = api.GetSearch(term=search_term,count=100,result_type='recent',return_json=True) #lang='en',
        search_tweets += query['statuses'] 
    # Keep getting tweets
    else:
        # Find the last tweet to use as a max_id
        max_id = search_tweets[-1]['id']
        # Get the next set of tweets
        query = api.GetSearch(term=search_term,count=100,return_json=True,max_id=max_id - 1) #lang='en',
        # Add them to the list of tweets
        search_tweets += query['statuses']   
    # When to stop?
    if len(search_tweets) > no_tweets:
        break
        
print("There are {0:,} tweets in the collection.".format(len(search_tweets)))

There are 2,100 tweets in the collection.


In [7]:
search_statuses_flat = []
search_errors = []

for i,status in enumerate(search_tweets):
    try:
        payload = tweet_cleaner(status)
        search_statuses_flat.append(payload)
    except:
        search_errors.append(str(i))

if len(search_errors) == 0:
    print("There were no errors!")
else:
    print("There were errors at the following indices:", '; '.join(search_errors[:50]))
    
search_df = pd.DataFrame(search_statuses_flat)
try:
    search_df['created'] = pd.to_datetime(search_df['created'])
#     search_df['created'] = search_df['created'].dt.tz_convert(None)
except KeyError as ex:
    print(ex)
search_df['created'] = search_df['created'].dt.tz_convert(None)
search_df.tail()

There were no errors!


,screen_name,created,retweets,favorites,id,reply_screen_name,reply_id,source,lang,place,user_mentions,hashtags,text,is_retweet,retweeted_screen_name,retweeted_created,retweeted_source
2095,esther_2U,2020-06-22 00:46:07,38,0,1274866221754662912,None,NaN,Twitter for iPhone,ko,None,mypreciousBogum,None,🖼🎨박보검 \n\n겨울에 올릴까 했던 거였는데 오늘🙂\n\n장소는 루브르미술관 Co...,True,mypreciousBogum,Sun Jun 21 13:09:57 +0000 2020,Twitter for iPhone
2096,regularhwang,2020-06-22 00:45:39,0,1,1274866103953563655,stigmataem,1.274864e+18,Twitter for iPhone,en,None,stigmataem,None,@stigmataem put these in the louvre,False,False,False,False
2097,Fragren36,2020-06-22 00:43:29,64,0,1274865556915654660,None,NaN,Twitter for Android,fr,None,MilovanCavor,Rosa; Louvre,"Avez-vous déjà vu une anoxie ? Celle-ci, salle...",True,MilovanCavor,Fri Jun 19 14:17:15 +0000 2020,Twitter for iPhone
2098,Pucci_en,2020-06-22 00:42:46,0,0,1274865379433566208,None,NaN,twittbot.net,en,None,None,None,Dio and I have the best sleepovers... Did you ...,False,False,False,False
2099,Yeezuss,2020-06-22 00:42:29,1,0,1274865307551698944,None,NaN,Twitter for iPhone,fr,None,nicolllettte; Yeezuss,None,@Yeezuss put it in the louvre,True,nicolllettte,Mon Jun 22 00:32:39 +0000 2020,Twitter for iPhone


In [8]:
# fdf=search_df[search_df['text'].astype(str).str.contains(search_term)] #"diabetes|Diabetes|DIABETES|diábetes"
# print(len(fdf))

In [9]:
stc=search_term.replace(" ","")
plname=stc+'_df.pic'
search_df.to_pickle(plname)

In [10]:
print("Run in %.2f seconds (%.2f minutes)" %(time.perf_counter() - start_time,(time.perf_counter() - start_time)/60))

Run in 13.15 seconds (0.22 minutes)
